**Video Game Ratings & Sales Analysis**

In this project, I'm exploring video game sales and ratings data to understand how the industry has evolved over time. With gaming projected to exceed $300 billion in global value by 2027, I want to dig deeper and see which games sold the most, which years produced the highest-rated titles, and whether there are certain “golden years” where critics and players were equally impressed.

The dataset I’m working with includes 400 top video games released since 1977. It contains both sales information and rating information (from critics and users). My job is to perform SQL queries inside the notebook to extract insights and save the results into specific DataFrames.

**1. Finding the Ten Best-Selling Games**

The first thing I want to examine is which games were the biggest commercial hits. To do that, I query the game_sales table, sort all games by the games_sold column in descending order, and pick the top ten.

I’m keeping all columns from the table because I want the full context—publisher, year, critic rating, everything.
Finally, I save this result into a DataFrame called best_selling_games.

**2. Identifying the Highest-Rated Years (Critics)**

Next, I want to see which years produced the best games according to critics.
But to avoid skewed results, I only consider years where at least four games were released.

So here, I:

Group games by year

Count how many games were released

Calculate the average critic score for each year

Only keep years with at least 4 games

Round the average critic score to two decimal places

Sort those years in descending order of critic score

Take the top ten

I save this query as a DataFrame named critics_top_ten_years.

(Important for myself: I am not allowed to use the critics_avg_year_rating helper table for this query.)

**3. Finding the “Golden Years” — When Critics and Users Agreed**

For my final analysis, I want to see when both critics and users broadly agreed that a specific year produced great games. In other words, I am looking for years where:

the average critic score > 9 OR

the average user score > 9

For this, I use the two pre-aggregated tables:

critics_avg_year_rating

users_avg_year_rating

I join these tables to combine critic and user averages for each year.
Then I compute a new column called diff to measure how much critics and users disagreed (critic score − user score).

In the final output, I include:

year

num_games

avg_critic_score

avg_user_score

diff

I sort everything by year (ascending) and save the result as golden_years.

In [1]:
-- best_selling_games
SELECT *
FROM game_sales
ORDER BY games_sold DESC
LIMIT 10;

,name,platform,publisher,developer,games_sold,year
0,Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
1,Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
2,Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
3,Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
4,PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
5,Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
6,Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
7,Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
8,New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
9,New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [2]:
-- critics_top_ten_years
SELECT g.year, COUNT(g.name) AS num_games, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM game_sales g
INNER JOIN reviews r
ON g.name = r.name
GROUP BY g.year
HAVING COUNT(g.name) >= 4
ORDER BY avg_critic_score DESC
LIMIT 10;

,year,num_games,avg_critic_score
0,1998,10,9.32
1,2004,11,9.03
2,2002,9,8.99
3,1999,11,8.93
4,2001,13,8.82
5,2011,26,8.76
6,2016,13,8.67
7,2013,18,8.66
8,2008,20,8.63
9,2017,13,8.62


In [3]:
-- golden_years
SELECT u.year, u.num_games, c.avg_critic_score, u.avg_user_score, c.avg_critic_score - u.avg_user_score AS diff
FROM critics_avg_year_rating c
INNER JOIN users_avg_year_rating u
ON c.year = u.year
WHERE c.avg_critic_score > 9 OR u.avg_user_score > 9
ORDER BY year ASC

,year,num_games,avg_critic_score,avg_user_score,diff
0,1997,8,7.93,9.50,-1.57
1,1998,10,9.32,9.40,-0.08
2,2004,11,9.03,8.55,0.48
3,2008,20,8.63,9.03,-0.40
4,2009,20,8.55,9.18,-0.63
5,2010,23,8.41,9.24,-0.83
